In [1]:
search_for = 133
start_from = 0
threads = 10
thread = 7
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.10942530632019043
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 385308361
type: [1, 1, 1, 1, 133] 133
cases of this type: 312900721
100000 18.28856603941487
200000 29.5625791640799
300000 29.122873444638326
400000 28.65870697295186
500000 28.73339752557072
600000 29.9436033864105
700000 29.101374727624744
800000 29.755261917001487
900000 28.419463108217613
1000000 28.329073990052695
1100000 28.135751686723175
1200000 12.472516074384302
1300000 24.258191136575626
1400000 27.974617975334386
1500000 28.360400195830394
1600000 28.044886668014577
1700000 28.308257835447364
1800000 29.971227752787684
1900000 28.51955667398489
2000000 28.224318153073757
2100000 28.45985740348396
2200000 28.793541480699858
2300000 29.138621824742035
2400000 7.707437561759737
2500000 30.501607031629185
2600000 36.22727686544798
2700000 35.5631700845157
2800000 35.38655482440006
2900000 35.64556930168129
3000000 34.657

29400000 19.178044820318338
29500000 16.88606759526162
29600000 26.359215370637166
29700000 27.52198411711963
29800000 27.67545172331082
29900000 28.837172868136438
30000000 27.534106608925384
30100000 28.840077967931364
30200000 29.124346909333887
30300000 28.467421338829595
30400000 12.54376217591502
30500000 29.452969135793378
30600000 29.546516680915204
30700000 30.70791322776582
30800000 26.31564986847266
30900000 15.238780756136704
31000000 29.600198091532242
31100000 29.914159266724862
31200000 29.225975461003618
31300000 28.586617962048095
31400000 28.37863135709208
31500000 28.740997804926607
31600000 27.654997425020294
31700000 27.012542516744766
31800000 7.997382149405471
31900000 27.985120545486343
32000000 27.284393661571947
32100000 27.518719812863257
32200000 28.173829804101977
32300000 28.925905552967503
32400000 28.593095459818834
32500000 29.973054488357977
32600000 29.1237915331317
32700000 19.73352179426522
32800000 22.034388448886904
32900000 29.319255730748967
330

59100000 27.436823455561647
59200000 27.161577944880506
59300000 11.453369195061224
59400000 24.89406574475889
59500000 25.466683667079746
59600000 25.745221466262212
59700000 23.539620812621
59800000 24.099318531572614
59900000 23.92942361973682
60000000 13.755043337816987
60100000 17.951201703480837
60200000 24.391185080180907
60300000 23.861859470379
60400000 23.984761318583377
60500000 25.137925962662795
60600000 25.068661174753377
60700000 21.277541519127347
60800000 17.09731116429349
60900000 27.189105909803626
61000000 27.07724933048949
61100000 26.595554215381014
61200000 26.954209718642208
61300000 26.909247370581827
61400000 26.746178169402498
61500000 26.950824461279126
61600000 25.59806424408766
61700000 10.275047521476603
61800000 25.222473751424026
61900000 25.419301744021176
62000000 24.124722564943255
62100000 24.32120283476268
62200000 23.71887904805079
62300000 24.18584651465782
62400000 6.207482756033318
62500000 24.322448001133772
62600000 23.2964445839203
62700000 

88900000 21.914490691157
89000000 23.200960531691074
89100000 22.982732567604543
89200000 24.27450060952339
89300000 22.64145650652852
89400000 24.23387744670719
89500000 23.922547728545222
89600000 23.889354070370924
89700000 23.912564324332635
89800000 23.35743318472048
89900000 22.729471552760835
90000000 21.92913529406486
90100000 12.037848326584466
90200000 17.819657929170596
90300000 20.689149272726432
90400000 22.217034643599842
90500000 21.286624367881476
90600000 9.40014917098762
90700000 18.931711216654342
90800000 21.360669601051068
90900000 20.993001523331856
91000000 21.049243899933373
91100000 10.838035820009958
91200000 20.135532633673712
91300000 22.590221163734448
91400000 21.991166813716585
91500000 22.700883041759194
91600000 22.831636965284556
91700000 23.82745892843179
91800000 24.215062429826492
91900000 23.481468860589374
92000000 23.79477417402528
92100000 24.470220391432612
92200000 23.338735160592393
92300000 22.369119626110404
92400000 21.906682154468918
9250

118000000 20.46385807300565
118100000 19.45488602722388
118200000 19.968168038197543
118300000 18.81298515228367
118400000 19.25981149517189
118500000 16.703035843115455
118600000 9.431499308534336
118700000 19.57302525395175
118800000 13.090545368380255
118900000 11.333398921057437
119000000 18.738645430981705
119100000 12.073511590169156
119200000 15.423772685422009
119300000 19.277878036135824
119400000 18.470710776496325
119500000 19.395206147307064
119600000 19.40575686070667
119700000 19.57268008099848
119800000 20.431085820346677
119900000 21.112424812401898
120000000 20.832690201727818
120100000 21.504538273281238
120200000 21.315871370339295
120300000 20.49361181365959
120400000 19.758613274075422
120500000 19.54630393230266
120600000 19.76279468876731
120700000 19.147211977755944
120800000 19.03555205566352
120900000 12.344266180993593
121000000 13.506211268636733
121100000 18.225323823140407
121200000 5.5330848961423
121300000 19.154625241099833
121400000 16.798095963248176


...

261400000 9.84896724753555
261500000 8.350108630072352
261600000 5.69805534681907
261700000 9.724533157248853
261800000 9.474345959068948
261900000 9.307276592847877
262000000 9.238045339813626
262100000 8.959601342888863
262200000 8.9241431150361
262300000 7.192884459472664
262400000 4.69084420845063
262500000 8.701661361005854
262600000 8.902751978934472
262700000 9.081762642594882
262800000 9.233436970779673
262900000 9.395536578240627
263000000 9.835836700420566
263100000 7.899184747412681
263200000 5.618186140749224
263300000 9.94929073282741
263400000 9.75201683412414
263500000 9.869732646989492
263600000 9.879560634428284
263700000 9.613469916238564
263800000 9.675875841575413
263900000 3.775175550406258
264000000 10.34738055431424
264100000 9.483224629885473
264200000 9.30928959388393
264300000 9.032372911028915
264400000 9.001508100829154
264500000 8.657068753631359
264600000 8.680972735510922
264700000 3.437843818998051
264800000 7.9974456298741
264900000 8.751533317078794
26

290800000 7.960892130537826
290900000 7.749899789450479
291000000 7.740881734040125
291100000 7.545334579981544
291200000 7.889258746906192
291300000 7.725197130933509
291400000 7.809530226393289
291500000 7.88279998361815
291600000 2.835089595484763
291700000 8.005445548658251
291800000 7.685635963981702
291900000 4.179631304382752
292000000 6.997318107678649
292100000 7.758195419248582
292200000 7.562879530758595
292300000 7.659134528954777
292400000 7.58936806471562
292500000 7.486081417472064
292600000 7.576371734496653
292700000 7.5247291838576
292800000 7.246779058389503
292900000 4.8046284745179975
293000000 4.685714443294126
293100000 7.230179012585854
293200000 7.486577678763214
293300000 7.426440596365887
293400000 7.550944422909919
293500000 7.753969778803313
293600000 7.655412799428975
293700000 7.38897497247031
293800000 7.687775737259696
293900000 7.186573954971042
294000000 3.612888052372972
294100000 7.486207517347254
294200000 6.047445917604568
294300000 5.040166229851

319700000 2.3976144372752706
319800000 2.437180005645195
319900000 2.557986902817585
320000000 2.716268106032157
320100000 2.4458770688873086
320200000 2.585335718782274
320300000 2.3190989999594165
320400000 2.3555131074458506
320500000 2.508680371479344
320600000 2.3459812743570154
320700000 2.0318048924810603
320800000 2.4128621112074238
320900000 2.1283030006736436
321000000 1.7223250124215417
321100000 1.3051866881092729
321200000 1.2570595582067874
321300000 1.1760753387783065
321400000 1.4680155300846993
321500000 2.3922321435755527
321600000 2.610635556852297
321700000 2.2104611464972357
321800000 1.9755391399571829
321900000 2.396065930424631
322000000 2.455563706486557
322100000 2.298902400556063
322200000 2.365392942691946
322300000 2.654387564183039
322400000 2.4495342475017012
322500000 2.428443261708011
322600000 2.3928094534464814
322700000 2.323110809578749
322800000 2.2732399382017796
322900000 2.0960285765440405
323000000 2.1223432730721616
323100000 2.513584769329935

348300000 1.5916467537719312
348400000 1.378322213043201
348500000 1.386233787751863
348600000 1.3575575483241333
348700000 1.407604743109268
348800000 1.144664176890168
348900000 1.2225449188607256
349000000 1.462010735412852
349100000 1.4134224977909533
349200000 0.9989834998375721
349300000 0.6868224852197529
349400000 0.7233960852447767
349500000 0.7455993867087828
349600000 0.9285210752012134
349700000 1.1281992017430431
349800000 1.4268640912579982
349900000 1.217114055411319
350000000 1.30437650287906
350100000 1.3250921482791105
350200000 1.2440825266160844
350300000 1.2657819392118315
350400000 1.4389652735839684
350500000 1.3450915822191458
350600000 1.4773536185345781
350700000 1.3393178925449336
350800000 1.349143239768223
350900000 1.257392725731474
351000000 1.2534213462862165
351100000 1.2330105903820814
351200000 1.2515556011804938
351300000 1.2235367768644552
351400000 1.2845366146699848
351500000 1.054191278314421
351600000 0.8638459242838177
351700000 0.7535397083294

376100000 0.059242254421037435
376200000 0.05917558265989555
376300000 0.05836649882116914
376400000 0.05835297501841717
best so far: 0
type: [1, 1, 7, 1, 19] 133
cases of this type: 6385729
376500000 0.284728742989099
376600000 0.36236152671598393
376700000 0.3483622867359068
376800000 0.3530113216371669
376900000 0.3118965157562395
377000000 0.3112104753580219
377100000 0.30275404965856206
377200000 0.2478282862239301
377300000 0.2446215462569972
377400000 0.24142919165440463
377500000 0.24296169379469754
377600000 0.24504997189429267
377700000 0.2355850861326019
377800000 0.23701753573904832
377900000 0.23541669762764372
378000000 0.22820008604911368
378100000 0.22973034260587097
378200000 0.23662288577825955
378300000 0.2340160347800056
378400000 0.2252024493697882
378500000 0.25979792670290075
378600000 0.2690260614130305
378700000 0.2600044855123778
378800000 0.2085728772823665
378900000 0.049020047682491276
379000000 0.04686403933609128
379100000 0.04631154909696579
379200000 0.